In [104]:
import json
import csv
from typing import Dict, List, Any, Optional
from datetime import datetime

def load_transport_data(json_path: str) -> List[Dict[str, Any]]:
    """승하차 정보 JSON 파일을 로드한다."""
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f"✅ 승하차 정보 {len(data)}건 로드 완료")
    return data


# 승하차 정보 데이터 로드
transport_data = load_transport_data("승하차정보.json")



✅ 승하차 정보 45건 로드 완료


In [105]:
import json
import pandas as pd
from typing import Dict, List, Any

def load_commute_allowance_data(json_path: str) -> List[Dict[str, Any]]:
    """통근 수당 JSON 파일을 로드한다."""
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(f"✅ 통근 수당 정보 {len(data)}건 로드 완료")
    return data


# 통근 수당 데이터 로드
commute_allowance_data = load_commute_allowance_data("통근수당.json")


✅ 통근 수당 정보 8건 로드 완료


In [106]:
import os
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# State 정의
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    transport_data: str
    commute_allowance_data: str
    analysis_result: str
    chart_type: str

# LLM 초기화
llm = ChatOpenAI(
    model="solar-pro2",
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)

# 노드 함수들
def chart_type_selector(state: GraphState):
    """차트 타입 선택 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-1].content
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가다. 
    
    사용자 질문: {user_question}
    
    알맞은 차트 타입을 선정해줘. 추가적인 내용 포함하지 말고, 차트 타입만 영어로 출력해줘. 
    선택 가능 차트: line_chart, bar_chart, text_summary, table
    """

    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "chart_type": response.content.strip(),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", ""),
        "analysis_result": ""
    }

def generate_analytic(state: GraphState):
    """인사이트 생성 노드"""
    user_question = ""
    if state["messages"]:
        user_question = state["messages"][-2].content  # 사용자 질문은 두 번째 마지막 메시지
    
    system_prompt = f"""
    너는 교통 데이터와 통근 수당 데이터를 분석하는 전문가야. python 코드로 작성하지말고 오직 분석만 작성해줘

    차트 타입 선택 결과: {state.get('chart_type', '')}  
    line chart 일경우 데이터 형식:
    
    교통 데이터: {state.get('transport_data', '')}
    통근 수당 데이터: {state.get('commute_allowance_data', '')}

    사용자 질문: {user_question}
    선택된 차트 타입: {state.get('chart_type', '')}
    
    
    위 데이터를 바탕으로 사용자 질문에 대한 분석 결과를 제공해줘.
    반드시 아래의 output format에 맞춰 JSON 형식으로만 출력한다. 다른 설명, 주석, 텍스트는 절대 포함하지 않는다. 오직 JSON 데이터만 출력한다.

    ```json ``` 안에 넣지마 그냥 아래 결과만 

    output format:
        {{
            "chart_data": {{
                "labels": ["January", "February", "March", "April", "May", "June", "July"],
                "datasets": [{{
                    "label": "My First Dataset",
                    "data": [65, 59, 80, 81, 56, 55, 40],
                    "fill": false,
                    "borderColor": "rgb(75, 192, 192)",
                    "tension": 0.1
                }}]
            }},
            "reason": "분석결과"
        }}
    """
    
    messages = [
        SystemMessage(content=system_prompt),
    ]
    
    response = llm.invoke(messages)
    
    return {
        "messages": state["messages"] + [response],
        "analysis_result": response.content,
        "chart_type": state.get("chart_type", ""),
        "transport_data": state.get("transport_data", ""),
        "commute_allowance_data": state.get("commute_allowance_data", "")
    }

# 그래프 구성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("chart_type_selector", chart_type_selector)
workflow.add_node("generate_analytic", generate_analytic)

# 엣지 설정
workflow.set_entry_point("chart_type_selector")
workflow.add_edge("chart_type_selector", "generate_analytic")
workflow.add_edge("generate_analytic", END)

# 그래프 컴파일
transport_analyzer = workflow.compile()

# 실행 예시
def run_transport_analysis(user_question: str = "", transport_data: str = "", commute_allowance_data: str = ""):
    """교통 데이터 분석 실행"""
    initial_state = {
        "messages": [HumanMessage(content=user_question)] if user_question else [],
        "transport_data": transport_data,
        "commute_allowance_data": commute_allowance_data,
        "analysis_result": "",
        "chart_type": ""
    }
    
    result = transport_analyzer.invoke(initial_state)
    return result

# 테스트 실행
# 실제 데이터를 문자열로 변환하여 전달
transport_data_str = str(transport_data)
commute_allowance_data_str = str(commute_allowance_data)

# 교통 데이터 분석 실행
analysis_result = run_transport_analysis(
    user_question="버스 노선 최대 수익율을 볼수 있는 라인 그래프 생성해줘",
    transport_data=transport_data_str,
    commute_allowance_data=commute_allowance_data_str
)

# # 분석 결과 출줘
# for message in analysis_result["messages"]:
#     if hasattr(message, 'content'):
#         print(f"\n{message.content}")


# 분석 결과 출력
if analysis_result["messages"]:
    last_message = analysis_result["messages"][-1]
    if hasattr(last_message, 'content'):
        print(f"\n{last_message.content}")


{
        "chart_data": {
                "labels": ["출근1호-한국대서문", "출근2호-한국전자기술연구원", "출근3호-판교공영주차장", "출근4호-Meta", "퇴근1호(전자-한양정형외과)", "퇴근2호(세파-한국대서문)", "퇴근3호(바이오-효성아파트)", "퇴근4호(바이오-선경아파트1차)"],
                "datasets": [{
                        "label": "수익율(%)",
                        "data": [13.698630137, 14.705882353, 11.494252874, 15.873015873, 11.494252874, 14.492753623, 18.181818182, 11.363636364],
                        "fill": false,
                        "borderColor": "rgb(75, 192, 192)",
                        "tension": 0.1
                }]
        },
        "reason": "통근 수당 데이터에서 '지급수당' 대비 '운행단가'의 비율을 계산하여 수익율(%)을 산출한 후, 각 노선별 수익율을 비교 분석할 수 있도록 라인 그래프를 구성하였습니다. 수익율 계산식: (지급수당 / 운행단가) * 100"
}
